# i) Regra da soma

In [14]:
import os
import numpy as np
import pandas as pd
import clustering

## Datasets

In [15]:
DATA_BASE_PATH = "./data"

FAC_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fac")
FOU_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fou")
KAR_FILE = os.path.join(DATA_BASE_PATH, "mfeat-kar")

fac = np.loadtxt(FAC_FILE, dtype=int)
fou = np.loadtxt(FOU_FILE, dtype=float)
kar = np.loadtxt(KAR_FILE, dtype=float)

## Importando melhor resultado

In [16]:
best_result = clustering.import_best_result("data/melhor_resultado_todas.pickle")
partition, y_true = clustering.get_hard_patitions(best_result["membership_degree"])


## Probabilidade à priori das classes

In [17]:
Pw = np.array([len(c)/2000 for c in partition])
Pw

array([0.1175, 0.116 , 0.1355, 0.1465, 0.0405, 0.01  , 0.1305, 0.151 ,
       0.0815, 0.071 ])

## Função de densidade

In [18]:
# def calc_gaussian_density_prob(xk, d, means, var, cov_matrix):
#     coef = np.power(2*np.pi, -d/2) 
#     inv_cov_matrix = np.linalg.inv(cov_matrix)
#     (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
#     sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
#     diff = xk - means
#     exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
#     exp_func = np.exp(exp_exp)
    
#     return coef * sqrt_det_inv_cov * exp_func

In [19]:

# def calc_gaussian_bayesian_data(x, partition):
#     n,d = x.shape
#     qtd_w = len(partition)
#     means = np.array([x[idxs].mean(axis=0) for idxs in partition])
#     var = np.array([((x[idxs]-means[i])**2).mean(axis=0) for i, idxs in enumerate(partition)])
#     cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

#     for i in range(qtd_w):
#         np.fill_diagonal(cov_matrix[i], var[i])
                
#     p_x_w = np.empty((n, qtd_w))
    
#     for k in range(n):
#         for i in range(qtd_w):
#             p_x_w[k, i] = calc_gaussian_density_prob(x[k], d, means[i], var[i], cov_matrix[i])   
    
#     return p_x_w

def calc_prob_posteriori(p_x_w, Pw):
    qtd_x, qtd_w = p_x_w.shape 
    p_w_x = np.empty((qtd_x, qtd_w))
    
    for k in range(qtd_x):
        for i in range(qtd_w):
            sum_all = np.dot(p_x_w[k], Pw)
            p_w_x[k,i] = (p_x_w[k,i] * Pw[i])/sum_all
       
    return p_w_x


## Densidades por dataset

In [20]:
# fac_byn_density_probs = calc_gaussian_bayesian_data(fac, partition)
# fou_byn_density_probs = calc_gaussian_bayesian_data(fou, partition)
# kar_byn_density_probs = calc_gaussian_bayesian_data(kar, partition)

In [21]:
# fac_byn_density_probs[0,0], fou_byn_density_probs[0,0], kar_byn_density_probs[0,0]

## Prob. à posteriori por view

In [22]:
# fac_byn_posteriori_probs = calc_prob_posteriori(fac_byn_density_probs, Pw)
# fou_byn_posteriori_probs = calc_prob_posteriori(fou_byn_density_probs, Pw)
# kar_byn_posteriori_probs = calc_prob_posteriori(kar_byn_density_probs, Pw)


## Regra da soma

Precisei tirar o fac porque seus valores são nulos. TENTAR CORRIGIR ISSO

In [23]:
def regra_soma_padrao(views, Pw):
    qtd_x = views[0].shape[0]
    qtd_w = len(Pw)
    x_sum_w = np.empty((qtd_x, qtd_w))

    for i in range(qtd_x):
        for k in range(qtd_w):
            views_sum = sum([v[i,k] for v in views])
            x_sum_w[i,k] = (1-len(views))*Pw[k] + views_sum
                    
    y_pred = x_sum_w.argmax(axis = 1) 
    return y_pred

## Estimador do scikit

In [24]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class ClassificaforBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
        self.partition = partition
        self.Pw = Pw
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self._fit_gaussian_bayesian_data(X)
        self.X_ = X
        self.y_ = y
        return self
    
    def _fit_gaussian_bayesian_data(self, X):
        n, d = X.shape
        qtd_w = len(self.partition)
        self.means = np.array([X[idxs].mean(axis=0) for idxs in self.partition])
        self.var = np.array([((X[idxs]-self.means[i])**2).mean(axis=0) for i, idxs in enumerate(self.partition)])
        self.cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

        for i in range(qtd_w):
            np.fill_diagonal(self.cov_matrix[i], self.var[i])

        return self
    
    def _calc_gaussian_density_prob(self, xk, cls):
        d = xk.shape[0]
        coef = np.power(2*np.pi, -d/2)
        inv_cov_matrix = np.linalg.inv(self.cov_matrix[cls])
        (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
        sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
        diff = xk - self.means[cls]
        exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
        exp_func = np.exp(exp_exp)

        return coef * sqrt_det_inv_cov * exp_func

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        
        desity_probs = np.empty((X.shape[0], len(self.classes_)))
        for k in range(desity_probs.shape[0]):
            for j in range(len(self.classes_)):
                desity_probs[k,j] = self._calc_gaussian_density_prob(X[k], j)
        
        post_probs = calc_prob_posteriori(desity_probs, self.Pw)
        
        return post_probs
        
        

In [25]:
class RegraSomaClasificadorBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
#         self.views = views
        self.partition = partition
        self.Pw = Pw
        self.clfs = []        
    
    def fit(self, X, y):
#         X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y
        
        for x in X:
            clf = ClassificaforBayesiano(self.partition, Pw)
            clf.fit(x,y)
            self.clfs.append(clf)
            
        return self
    
    def predict(self, X):
        assert len(X) == len(self.clfs)
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
#         X = check_array(X)
        
        post_probs = [clf.predict_proba(x) for clf, x in zip(self.clfs, X)]
        
        return self.regra_soma(post_probs, Pw=self.Pw)
    
    def get_params(self, deep=True):
        return {"Pw": self.Pw, "partition": self.partition}
    
    def regra_soma(self, matrizes, Pw):
        qtd_x = matrizes[0].shape[0]
        qtd_w = len(Pw)
        x_sum_w = np.empty((qtd_x, qtd_w))
        
        for i in range(qtd_x):
            for k in range(qtd_w):
                views_sum = sum([v[i,k] for v in matrizes])
                # views_sum =  fac_p_w_x[i,k] fou_p_w_x[i,k] + kar_p_w_x[i,k]
                x_sum_w[i,k] = (1-len(matrizes))*Pw[k] + views_sum
                
        y_pred = x_sum_w.argmax(axis = 1) 
        return y_pred

# def regra_soma_padrao(fac_p_w_x, fou_p_w_x, kar_p_w_x, Pw):
# x_sum_w = np.empty((10, 2000))

# for i in range(10):
#     for k in range(2000):
#         views_sum =  fou_p_w_x[i,k] + kar_p_w_x[i,k]
#         # views_sum =  fac_p_w_x[i,k] fou_p_w_x[i,k] + kar_p_w_x[i,k]
#         x_sum_w[i,k] = (1-2)*Pw[i] + views_sum

# y_pred = x_sum_w.argmax(axis = 0) 
# return y_pred

## Validação cruzada

- **Observação**: No modelo gaussiano,  as desidades de probabilidades da VIEW1 (mfeat-fac) são todas zeradas, o que causa uma baixa na performance geral do modelo da regra da soma. Por esta razão, apenas neste modelo, ela foi desconsiderada. Portanto, no modelo gaussiano apresentamos os resultados os dados considerando somente VIEW2 e VIEW3.

In [26]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_fscore_support


RANDOM_SEED = 42
FOLDS = 10
REPEATS = 30

def get_splited_partition(idxs, y_true):
    partition = [[] for i in range(10)]
    
    for i,indice in enumerate(idxs):
        partition[y_true[indice]].append(i)
        
    return partition
    
cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)

train_precision_scores = np.empty((cv.get_n_splits(),))
train_recall_scores = np.empty((cv.get_n_splits(),))
train_f1_scores = np.empty((cv.get_n_splits(),))

test_precision_scores = np.empty((cv.get_n_splits(),))
test_recall_scores = np.empty((cv.get_n_splits(),))
test_f1_scores = np.empty((cv.get_n_splits(),))

#views = [fac, fou, kar]
views = [fou, kar]

for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
    print(f"{i+1}/{cv.get_n_splits()}", end=" ")
    
    test_views = [v[test_index] for v in views]
    train_views = [v[train_index] for v in views]
    
    local_partition = get_splited_partition(train_index, y_true)
    clf = RegraSomaClasificadorBayesiano(local_partition, Pw)
    
    clf.fit(train_views, y_true[train_index])
    
    y_pred_train = clf.predict(train_views)
    y_pred_test = clf.predict(test_views)
    
    scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")

    train_precision_scores[i] = scores_train[0]
    train_recall_scores[i] = scores_train[1]
    train_f1_scores[i] = scores_train[2]

    scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                   y_pred_test, 
                                                   average="macro")

    test_precision_scores[i] = scores_test[0]
    test_recall_scores[i] = scores_test[1]
    test_f1_scores[i] = scores_test[2]

    report = dict(
        runs = cv.get_n_splits(),
        train_precision = train_precision_scores.mean(),
        train_precision_std = train_precision_scores.std(),
        train_recall = train_recall_scores.mean(),
        train_recall_std = train_recall_scores.std(),
        train_fscore = train_f1_scores.mean(),
        train_fscore_std = train_f1_scores.std(),
        test_precision = test_precision_scores.mean(),
        test_precision_std = test_precision_scores.std(),
        test_recall = test_recall_scores.mean(),
        test_recall_std = test_recall_scores.std(),
        test_fscore = test_f1_scores.mean(),
        test_fscore_std = test_f1_scores.std(),

    )

    if os.path.exists("data/gaussian_training_data.csv"):
        pd.DataFrame([report]).to_csv("data/gaussian_training_data.csv", mode="a", header=False, decimal=",", index=False)
    else:
        pd.DataFrame([report]).to_csv("data/gaussian_training_data.csv", mode="a", header=True, decimal=",", index=False)



1/300 

/home/thiago/.anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


2/300 

/home/thiago/.anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


3/300 

...

297/300 

/home/thiago/.anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


298/300 

/home/thiago/.anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


299/300 300/300 

# ii) K-Vizinhos

## Normalizando 


In [27]:
from sklearn.preprocessing import minmax_scale

fac_norm = minmax_scale(fac)
fou_norm = minmax_scale(fou)
kar_norm = minmax_scale(kar)

## Distâncias entre os elementos

In [28]:
from sklearn.metrics.pairwise import euclidean_distances

fac_dist = euclidean_distances(fac_norm, fac_norm)
fou_dist = euclidean_distances(fou_norm, fou_norm)
kar_dist = euclidean_distances(kar_norm, kar_norm)

## Função de densidade

In [29]:
def calc_knn_density_prob(view_dists, k, Pw, y_true):
    qtd_x = view_dists.shape[0]
    qtd_w = len(Pw)
    
    p_x_w = np.empty((qtd_x, qtd_w))
    k_vizinhos = view_dists.argsort(axis=1)[:,:k+1]
    
    for j in range(qtd_x):
        w_vizinhos = y_true[k_vizinhos[j, 1:]]
        for i in range(qtd_w):
            p_x_w[j,i] = (w_vizinhos == i).sum()/k
                    
    return p_x_w
    

## Desidades por dataset

In [30]:
# fac_knn_density_probs = calc_knn_density_prob(fac_dist, 5, Pw, y_true)
# fou_knn_density_probs = calc_knn_density_prob(fou_dist, 5, Pw, y_true)
# kar_knn_density_probs = calc_knn_density_prob(kar_dist, 5, Pw, y_true)


## Prob. à posteriori por view

In [31]:
# fac_knn_posteriori_probs = calc_prob_posteriori(fac_knn_density_probs, Pw)
# fou_knn_posteriori_probs = calc_prob_posteriori(fou_knn_density_probs, Pw)
# kar_knn_posteriori_probs = calc_prob_posteriori(kar_knn_density_probs, Pw)

## Regra da soma

In [32]:
# y_pred_knn_all = regra_soma_padrao([fac_knn_posteriori_probs, 
#                                    fou_knn_posteriori_probs, 
#                                    kar_knn_posteriori_probs], Pw)

# print("Acurácia: ", accuracy_score(y_true, y_pred_knn_all))
# print("Medida-F: ", f1_score(y_true, y_pred_knn_all, average="macro"))

## Validação cruzada

In [33]:
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import os

cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)


k_range = range(3,16,2)
views = [fac, fou, kar]

for k in k_range:
    
    train_precision_scores = np.empty((cv.get_n_splits(),))
    train_recall_scores = np.empty((cv.get_n_splits(),))
    train_f1_scores = np.empty((cv.get_n_splits(),))

    test_precision_scores = np.empty((cv.get_n_splits(),))
    test_recall_scores = np.empty((cv.get_n_splits(),))
    test_f1_scores = np.empty((cv.get_n_splits(),))
    
    for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
        print(f"[k={k}] {i+1}/{cv.get_n_splits()}", end=" ")

        train_views = [minmax_scale(v[train_index]) for v in views]
        test_views = [minmax_scale(v[test_index]) for v in views]

        train_views_dists = [euclidean_distances(v,v) for v in train_views]
        test_views_dists = [euclidean_distances(v,v) for v in test_views]

    #     y_true_split = get_splited_partition(train_index, y_true)

        train_views_density_probs = [calc_knn_density_prob(v, k, Pw, y_true[train_index]) 
                                     for v in train_views_dists]

        train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in train_views_density_probs]

        y_pred_train = regra_soma_padrao(train_views_post_probs, Pw)


        test_views_density_probs = [calc_knn_density_prob(v, k, Pw, y_true[test_index]) 
                                     for v in test_views_dists]

        test_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in test_views_density_probs]

        y_pred_test = regra_soma_padrao(test_views_post_probs, Pw)
        
        scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")
        
        train_precision_scores[i] = scores_train[0]
        train_recall_scores[i] = scores_train[1]
        train_f1_scores[i] = scores_train[2]

        scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                       y_pred_test, 
                                                       average="macro")
        
        test_precision_scores[i] = scores_test[0]
        test_recall_scores[i] = scores_test[1]
        test_f1_scores[i] = scores_test[2]
        
    report = dict(
        k = k,
        runs = cv.get_n_splits(),
        train_precision = train_precision_scores.mean(),
        train_precision_std = train_precision_scores.std(),
        train_recall = train_recall_scores.mean(),
        train_recall_std = train_recall_scores.std(),
        train_fscore = train_f1_scores.mean(),
        train_fscore_std = train_f1_scores.std(),
        test_precision = test_precision_scores.mean(),
        test_precision_std = test_precision_scores.std(),
        test_recall = test_recall_scores.mean(),
        test_recall_std = test_recall_scores.std(),
        test_fscore = test_f1_scores.mean(),
        test_fscore_std = test_f1_scores.std(),
        
    )
    
    if os.path.exists("data/knn_best_k.csv"):
        pd.DataFrame([report]).to_csv("data/knn_best_k.csv", mode="a", header=False, decimal=",", index=False)
    else:
        pd.DataFrame([report]).to_csv("data/knn_best_k.csv", mode="a", header=True, decimal=",", index=False)
    
        
        
        
#         if test_f1_scores[:i+1].mean() > best_data["f1_scores"].mean():
#             best_data["f1_scores"] = test_f1_scores[:i+1]
#             best_data["k"] = k
            
#             print(f"\nMelhor k: {k}")
#             print(f"Acurácia de treino parcial: {train_acc_scores[:i+1].mean():4} +/- ({train_acc_scores[:i+1].std():4})")
#             print(f"Medida-F de treino parcial: {train_f1_scores[:i+1].mean():4} +/- ({train_f1_scores[:i+1].std():4})\n")

#             print(f"Acurácia de test parcial: {test_acc_scores[:i+1].mean():4} +/- ({test_acc_scores[:i+1].std():4})")
#             print(f"Medida-F de test parcial: {test_f1_scores[:i+1].mean():4} +/- ({test_f1_scores[:i+1].std():4})\n")

    
    

[k=3] 1/300 

/home/thiago/.anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[k=3] 2/300 

/home/thiago/.anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[k=3] 3/300 

...

/home/thiago/.anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[k=15] 299/300 

/home/thiago/.anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[k=15] 300/300 

/home/thiago/.anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# iii) Janela de Parzen

## Função de densidade

In [34]:
def parzen_density_function(view, h, partition):
    qtd_x = view.shape[0]
    qtd_w = len(partition)
    
    p_x_w = np.empty((qtd_x, qtd_w))
    dims = view.shape[1]
    
    for i in range(qtd_w):
        n = len(partition[i])
        x_view = view[partition[i],:]
        for k in range(qtd_x):
            diff = (view[k] - x_view)/h
            gaussian_kernel = np.exp(-(diff**2)/2)/np.sqrt(2*np.pi)
            prod_dims = gaussian_kernel.prod(axis=1)
            p_x_w[k,i] = prod_dims.sum()/(n*h**dims)
            
    return p_x_w

In [35]:
fac_parzen_density_probs = parzen_density_function(fac, 2, partition)
fou_parzen_density_probs = parzen_density_function(fou, 2, partition)
kar_parzen_density_probs = parzen_density_function(kar, 2, partition)

## Prob. à posteriori por view

In [36]:
fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

## Regra da soma

In [37]:
# for h in range(2, 21, 1):
#     fac_parzen_density_probs = parzen_density_function(fac, h, partition)
#     fou_parzen_density_probs = parzen_density_function(fou, h, partition)
#     kar_parzen_density_probs = parzen_density_function(kar, h, partition)

#     fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
#     fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
#     kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

#     y_pred_prazen_all = regra_soma_padrao([fac_prazen_posteriori_probs, 
#                                        fou_prazen_posteriori_probs, 
#                                        kar_prazen_posteriori_probs], Pw)

#     print(f"h: {h}")
#     print("Acurácia: ", accuracy_score(y_true, y_pred_prazen_all))
#     print("Medida-F: ", f1_score(y_true, y_pred_prazen_all, average="macro"))

## Validação cruzada

In [ ]:
cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)
train_acc_scores = np.empty((cv.get_n_splits(),))
train_f1_scores = np.empty((cv.get_n_splits(),))

test_acc_scores = np.empty((cv.get_n_splits(),))
test_f1_scores = np.empty((cv.get_n_splits(),))


h_range = range(2,21,2)
best_data = {"f1_scores": np.zeros((1,)), 
             "h":None}

views = [fac, fou, kar]

y_true = y_true.astype(int)
for h in h_range:
    train_precision_scores = np.empty((cv.get_n_splits(),))
    train_recall_scores = np.empty((cv.get_n_splits(),))
    train_f1_scores = np.empty((cv.get_n_splits(),))

    test_precision_scores = np.empty((cv.get_n_splits(),))
    test_recall_scores = np.empty((cv.get_n_splits(),))
    test_f1_scores = np.empty((cv.get_n_splits(),))
    
    for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
        print(f"[h={h}] {i+1}/{cv.get_n_splits()}", end=" ")

        train_views = [v[train_index] for v in views]
        test_views = [v[test_index] for v in views]

        
        y_true_split = get_splited_partition(train_index, y_true)
        

        train_views_density_probs = [parzen_density_function(v, h, y_true_split) 
                                     for v in train_views]

        train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in train_views_density_probs]

        y_pred_train = regra_soma_padrao(train_views_post_probs, Pw)


        y_true_split = get_splited_partition(test_index, y_true)
        
        test_views_density_probs = [parzen_density_function(v, h, y_true_split) 
                                     for v in test_views]

        test_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in test_views_density_probs]

        y_pred_test = regra_soma_padrao(test_views_post_probs, Pw)

        scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")

        train_precision_scores[i] = scores_train[0]
        train_recall_scores[i] = scores_train[1]
        train_f1_scores[i] = scores_train[2]

        scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                       y_pred_test, 
                                                       average="macro")

        test_precision_scores[i] = scores_test[0]
        test_recall_scores[i] = scores_test[1]
        test_f1_scores[i] = scores_test[2]

    report = dict(
        h = h,
        runs = cv.get_n_splits(),
        train_precision = train_precision_scores.mean(),
        train_precision_std = train_precision_scores.std(),
        train_recall = train_recall_scores.mean(),
        train_recall_std = train_recall_scores.std(),
        train_fscore = train_f1_scores.mean(),
        train_fscore_std = train_f1_scores.std(),
        test_precision = test_precision_scores.mean(),
        test_precision_std = test_precision_scores.std(),
        test_recall = test_recall_scores.mean(),
        test_recall_std = test_recall_scores.std(),
        test_fscore = test_f1_scores.mean(),
        test_fscore_std = test_f1_scores.std(),

    )

    if os.path.exists("data/parzen_best_h.csv"):
        pd.DataFrame([report]).to_csv("data/parzen_best_h.csv", mode="a", header=False, decimal=",", index=False)
    else:
        pd.DataFrame([report]).to_csv("data/parzen_best_h.csv", mode="a", header=True, decimal=",", index=False)
    
    

[h=2] 1/300 [h=2] 2/300 [h=2] 3/300 [h=2] 4/300 [h=2] 5/300 [h=2] 6/300 [h=2] 7/300 [h=2] 8/300 [h=2] 9/300 [h=2] 10/300 [h=2] 11/300 [h=2] 12/300 [h=2] 13/300 [h=2] 14/300 [h=2] 15/300 [h=2] 16/300 [h=2] 17/300 [h=2] 18/300 [h=2] 19/300 [h=2] 20/300 [h=2] 21/300 [h=2] 22/300 [h=2] 23/300 [h=2] 24/300 [h=2] 25/300 [h=2] 26/300 [h=2] 27/300 [h=2] 28/300 [h=2] 29/300 [h=2] 30/300 [h=2] 31/300 [h=2] 32/300 [h=2] 33/300 [h=2] 34/300 [h=2] 35/300 [h=2] 36/300 [h=2] 37/300 [h=2] 38/300 [h=2] 39/300 [h=2] 40/300 [h=2] 41/300 [h=2] 42/300 [h=2] 43/300 [h=2] 44/300 [h=2] 45/300 [h=2] 46/300 [h=2] 47/300 [h=2] 48/300 [h=2] 49/300 [h=2] 50/300 [h=2] 51/300 [h=2] 52/300 [h=2] 53/300 [h=2] 54/300 [h=2] 55/300 [h=2] 56/300 [h=2] 57/300 [h=2] 58/300 [h=2] 59/300 [h=2] 60/300 [h=2] 61/300 [h=2] 62/300 [h=2] 63/300 [h=2] 64/300 [h=2] 65/300 [h=2] 66/300 [h=2] 67/300 [h=2] 68/300 [h=2] 69/300 [h=2] 70/300 [h=2] 71/300 [h=2] 72/300 [h=2] 73/300 [h=2] 74/300 [h=2] 75/300 [h=2] 76/300 [h=2] 77/300 [h=2] 78

[h=6] 2/300 [h=6] 3/300 [h=6] 4/300 [h=6] 5/300 [h=6] 6/300 [h=6] 7/300 [h=6] 8/300 [h=6] 9/300 [h=6] 10/300 [h=6] 11/300 [h=6] 12/300 [h=6] 13/300 [h=6] 14/300 [h=6] 15/300 [h=6] 16/300 [h=6] 17/300 [h=6] 18/300 [h=6] 19/300 [h=6] 20/300 [h=6] 21/300 [h=6] 22/300 [h=6] 23/300 [h=6] 24/300 [h=6] 25/300 [h=6] 26/300 [h=6] 27/300 [h=6] 28/300 [h=6] 29/300 [h=6] 30/300 [h=6] 31/300 [h=6] 32/300 [h=6] 33/300 [h=6] 34/300 [h=6] 35/300 [h=6] 36/300 [h=6] 37/300 [h=6] 38/300 [h=6] 39/300 [h=6] 40/300 [h=6] 41/300 [h=6] 42/300 [h=6] 43/300 [h=6] 44/300 [h=6] 45/300 [h=6] 46/300 [h=6] 47/300 [h=6] 48/300 [h=6] 49/300 [h=6] 50/300 [h=6] 51/300 [h=6] 52/300 [h=6] 53/300 [h=6] 54/300 [h=6] 55/300 [h=6] 56/300 [h=6] 57/300 [h=6] 58/300 [h=6] 59/300 [h=6] 60/300 [h=6] 61/300 [h=6] 62/300 [h=6] 63/300 [h=6] 64/300 [h=6] 65/300 [h=6] 66/300 [h=6] 67/300 [h=6] 68/300 [h=6] 69/300 [h=6] 70/300 [h=6] 71/300 [h=6] 72/300 [h=6] 73/300 [h=6] 74/300 [h=6] 75/300 [h=6] 76/300 [h=6] 77/300 [h=6] 78/300 [h=6] 7

[h=10] 3/300 [h=10] 4/300 [h=10] 5/300 [h=10] 6/300 [h=10] 7/300 [h=10] 8/300 [h=10] 9/300 [h=10] 10/300 [h=10] 11/300 [h=10] 12/300 [h=10] 13/300 [h=10] 14/300 [h=10] 15/300 [h=10] 16/300 [h=10] 17/300 [h=10] 18/300 [h=10] 19/300 [h=10] 20/300 [h=10] 21/300 [h=10] 22/300 [h=10] 23/300 [h=10] 24/300 [h=10] 25/300 [h=10] 26/300 [h=10] 27/300 [h=10] 28/300 [h=10] 29/300 [h=10] 30/300 [h=10] 31/300 [h=10] 32/300 [h=10] 33/300 [h=10] 34/300 [h=10] 35/300 [h=10] 36/300 [h=10] 37/300 [h=10] 38/300 [h=10] 39/300 [h=10] 40/300 [h=10] 41/300 [h=10] 42/300 [h=10] 43/300 [h=10] 44/300 [h=10] 45/300 [h=10] 46/300 [h=10] 47/300 [h=10] 48/300 [h=10] 49/300 [h=10] 50/300 [h=10] 51/300 [h=10] 52/300 [h=10] 53/300 [h=10] 54/300 [h=10] 55/300 [h=10] 56/300 [h=10] 57/300 [h=10] 58/300 [h=10] 59/300 [h=10] 60/300 [h=10] 61/300 [h=10] 62/300 [h=10] 63/300 [h=10] 64/300 [h=10] 65/300 [h=10] 66/300 [h=10] 67/300 [h=10] 68/300 [h=10] 69/300 [h=10] 70/300 [h=10] 71/300 [h=10] 72/300 [h=10] 73/300 [h=10] 74/300